In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from collections import Counter
import scipy.stats

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')
df = df.sort_values('timestamp')
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')

take last 2 months for validation  
remove gate_id that I don't want to predict by model (I predicted it based on EDA)

In [3]:
df_train = df[df['timestamp'] < pd.to_datetime('2022-11-01')]
df_test = df[df['timestamp'] >= pd.to_datetime('2022-11-01')]
df_test = df_test[~(df_test['gate_id'].isin([14,1,-1]))]

In [4]:
def get_feats(df):
    # time features
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    df['second'] = df['timestamp'].dt.second
    df['date'] = df['timestamp'].apply(lambda x: str(x)[:10])
    df['start_month'] = (df["timestamp"].dt.is_month_start).astype('int') 
    df['end_month'] = (df["timestamp"].dt.is_month_end).astype('int') 
    # get previous timestamps
    df['timestamp_prev'] = df['timestamp'].shift(1)
    df['timestamp_prev2'] = df['timestamp'].shift(2)
    df['timestamp_prev3'] = df['timestamp'].shift(3)
    df['timestamp_prev4'] = df['timestamp'].shift(4)
    df['timestamp_prev5'] = df['timestamp'].shift(5)
    # get previous gates
    df['prev_gate_id'] = df['gate_id'].shift(1).fillna(-2)
    df['prev2_gate_id'] = df['gate_id'].shift(2).fillna(-2)
    df['prev3_gate_id'] = df['gate_id'].shift(3).fillna(-2)
    df['prev4_gate_id'] = df['gate_id'].shift(4).fillna(-2)
    df['prev5_gate_id'] = df['gate_id'].shift(5).fillna(-2)
    # get min, max, first, last, moda and cnt gates        
    tmp = df.groupby(['date'])['gate_id'].min().reset_index()
    tmp=tmp.rename({'gate_id':'min_gate_id'},axis=1)
    df = df.merge(tmp, how='left', on=['date'])
    tmp = df.groupby(['date'])['gate_id'].max().reset_index()
    tmp=tmp.rename({'gate_id':'max_gate_id'},axis=1)
    df = df.merge(tmp, how='left', on=['date'])
    tmp = df.groupby(['date'])['gate_id'].first().reset_index()
    tmp=tmp.rename({'gate_id':'first_gate_id'},axis=1)
    df = df.merge(tmp, how='left', on=['date'])
    tmp = df.groupby(['date'])['gate_id'].last().reset_index()
    tmp=tmp.rename({'gate_id':'last_gate_id'},axis=1)
    df = df.merge(tmp, how='left', on=['date'])
    tmp = df.groupby(['date'])['gate_id'].agg(lambda x: x.value_counts().index[0]).reset_index()
    tmp=tmp.rename({'gate_id':'moda_gate_id'},axis=1)
    df = df.merge(tmp, how='left', on=['date'])
    df['min_gate_id'] = (df['gate_id'] == df['min_gate_id']).astype(int)
    df['max_gate_id'] = (df['gate_id'] == df['max_gate_id']).astype(int)
    df['first_gate_id'] = (df['gate_id'] == df['first_gate_id']).astype(int)
    df['last_gate_id'] = (df['gate_id'] == df['last_gate_id']).astype(int)
    df['moda_gate_id'] = (df['gate_id'] == df['moda_gate_id']).astype(int)
    tmp = df.groupby(['date','gate_id']).size().reset_index()
    tmp=tmp.rename({0:'cnt_gateday'},axis=1)
    df = df.merge(tmp, how='left', on=['date','gate_id'])
    # get same gates with previous
    df['same_gate_id'] = (df['gate_id'] == df['prev_gate_id']).astype('int')   
    df['same2_gate_id'] = (df['gate_id'] == df['prev2_gate_id']).astype('int')   
    df['same3_gate_id'] = (df['gate_id'] == df['prev3_gate_id']).astype('int')
    df['same4_gate_id'] = (df['gate_id'] == df['prev4_gate_id']).astype('int')   
    df['same5_gate_id'] = (df['gate_id'] == df['prev5_gate_id']).astype('int') 
    # get delta timestamps
    df['delta'] = df['timestamp_prev'] - df['timestamp']
    df['delta2'] = df['timestamp_prev2'] - df['timestamp']
    df['delta3'] = df['timestamp_prev3'] - df['timestamp']
    df['delta4'] = df['timestamp_prev4'] - df['timestamp']
    df['delta5'] = df['timestamp_prev5'] - df['timestamp']
    df['delta_days'] = df['delta'].dt.days.fillna(0)
    df['delta_hours'] = (df['delta'].dt.seconds // 3600).fillna(0)
    df['delta_minutes'] = (df['delta'].dt.seconds // 60 - (df['delta'].dt.seconds // 3600 * 60)).fillna(0)
    df['delta_seconds'] = (df['delta'].dt.seconds).fillna(0)
    df['delta_seconds2'] = (df['delta2'].dt.seconds).fillna(0)
    df['delta_seconds3'] = (df['delta3'].dt.seconds).fillna(0)
    df['delta_seconds4'] = (df['delta4'].dt.seconds).fillna(0)
    df['delta_seconds5'] = (df['delta5'].dt.seconds).fillna(0)
    df['delta_seconds_0'] = (df['delta_seconds'] == 0).astype(int)
    df['delta_seconds_3'] = (df['delta_seconds'] <= 3).astype(int)
    df['delta_seconds_86000'] = (df['delta_seconds'] > 86000).astype(int) # a lot of counts with this number of seconds
    df['delta_seconds3_86000'] = (df['delta_seconds3'] > 86000).astype(int) 
    df['morning'] = ((df['hour'] >= 6) & (df['hour'] < 12)).astype(int)
    df['evening'] = ((df['hour'] >= 18) & (df['hour'] <= 23)).astype(int)
    df['minute15'] = (df['minute'] <= 15).astype(int)
    df['minute45'] = (df['minute'] >= 45).astype(int)
    df['delta_minutes45'] = (df['delta_minutes'] > 45).astype(int)
    df['2days'] = (df["timestamp"].dt.dayofyear % 4).apply(lambda x: 1 if x in (1, 2) else 0) # 2-2 work
    # get other timestamp features
    df['day'] = df['timestamp'].dt.day
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['last_week'] = (df['day'] >= 30).astype('int') 
    df['first_week'] = (df['day'] <= 5).astype('int')       
    df['weekend'] = df['dayofweek'].isin([6,7]).astype('int')
    df['weekend'] = np.where(df['date'] == '2022-11-05',0,df['weekend'])
    df['holiday'] = (df['date'].isin(['2022-09-12','2022-11-25','2023-01-16'])).astype(int)
    df['holiday'] = np.where(df['date'] > '2023-02-02',1,df['holiday'])
    df['holiday'] = np.where(((df['date'] > '2022-11-07') & (df['date'] < '2022-11-17')).astype(int),1,df['holiday'])
    df['weekend'] = (df['weekend'] | df['holiday']).astype(int) # union holiday and weekend
    enc = OneHotEncoder()
    enc.fit(df[['dayofweek']])
    days = pd.DataFrame(enc.transform(df[['dayofweek']]).toarray())
    days.columns = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    days.index = df.index
    df = df.join(days)
    # get gates patterns
    df['3_4_7_10_11'] = (df['gate_id'].isin([3,4,7,10,11])).astype(int)
    df['4_5_7_10_11'] = (df['gate_id'].isin([5,4,7,10,11])).astype(int)
    df['3_4'] = ((df['gate_id'] == 3) & (df['prev_gate_id'] == 4)).astype(int)
    df['10_11'] = ((df['gate_id'] == 10) & (df['prev_gate_id'] == 11)).astype(int)
    df['4_5'] = ((df['gate_id'] == 4) & (df['prev_gate_id'] == 5)).astype(int)
    # get binary for each gate, hour, prev_gate, delta_hour   
    for i in range(-1,17):
        df[f'gate_{i}'] = (df['gate_id'] == i).astype('int')        
    for i in range(7,24):
        df[f'hour_{i}'] = (df['hour'] == i).astype('int')
    for i in range(-1,17):
        df[f'prev_gate_{i}'] = (df['prev_gate_id'] == i).astype('int') 
    for i in range(-1,17):
        df[f'prev2_gate_{i}'] = (df['prev2_gate_id'] == i).astype('int') 
    for i in range(24):
        df[f'delta_hours_{i}'] = (df['delta_hours'] == i).astype('int')
    # get more timestamp lags          
    df["lag"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if x < 2 else 0)
    df["l3s"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if x <= 3 else 0)
    df["l-1"] = df["l3s"].shift(-1)
    df.loc[(df["l3s"] == 1) | (df["l-1"] == 1), "ls1"] = 1
    df["lag1"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if 6 > x > 2 else 0)
    df["lag2"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if 15 > x >= 6 else 0)
    df["lag3"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if 22 > x >= 15 else 0)
    df["lag4"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if 32 > x >= 22 else 0)
    df["lag5"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if 42 > x > 32 else 0)
    df["lag6"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if 58 > x >= 42 else 0)
    df["lag7"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if 69 > x >= 58 else 0)
    df["lag8"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if 76 > x >= 69 else 0)
    df["lag9"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if 130 >= x > 127 else 0)
    df["lag0"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 0 if x > 69 else x)
    df["lagx"] = ((df["timestamp"] - df["timestamp_prev"]) / np.timedelta64(1, "s")).apply(lambda x: 1 if x > 130 else 0)
    # get more same gates
    df["double"] = ((df["gate_id"] == df['prev_gate_id']) & (df["ls1"] == 1) | (df["gate_id"] == df['prev2_gate_id'])).astype('int') 
    df["double2"] = ((df["gate_id"] == df["prev_gate_id"]) & (df["lag"] == 0) | (df["gate_id"] == df["prev2_gate_id"])).astype('int')
    df["double3"] = ((df["gate_id"] == df["prev_gate_id"]) & (df["lag1"] == 1) | (df["gate_id"] == df["prev2_gate_id"])).astype('int')
    df["double4"] = ((df["gate_id"] == df["prev_gate_id"]) & (df["lag2"] == 1) | (df["gate_id"] == df["prev2_gate_id"])).astype('int')
    df["double5"] = ((df["gate_id"] == df["prev_gate_id"]) & (df["lag3"] == 1) | (df["gate_id"] == df["prev2_gate_id"])).astype('int')
    df["double6"] = ((df["gate_id"] == df["prev_gate_id"]) & (df["lag4"] == 1) | (df["gate_id"] == df["prev2_gate_id"])).astype('int')
    df["double7"] = ((df["gate_id"] == df["prev_gate_id"]) & (df["lag7"] == 1) | (df["gate_id"] == df["prev2_gate_id"])).astype('int')
    df["double8"] = ((df["gate_id"] == df["prev_gate_id"]) & (df["lagx"] == 1) | (df["gate_id"] == df["prev2_gate_id"])).astype('int')
    # get more gate patterns
    # 10-3-3
    df.loc[((df["gate_id"]==10) & (df['prev_gate_id']==3) & (df['prev2_gate_id']==3)) |
        ((df["gate_id"]==10) & (df['prev_gate_id']==3) & (df['prev3_gate_id']==3)) |
        ((df["gate_id"]==10) & (df['prev2_gate_id']==3) & (df['prev3_gate_id']==3)) |
        ((df["gate_id"]==10) & (df['prev_gate_id']==3) & (df['prev4_gate_id']==3)) |
        ((df["gate_id"]==10) & (df['prev2_gate_id']==3) & (df['prev4_gate_id']==3)) |
        ((df["gate_id"]==10) & (df['prev_gate_id']==3) & (df['prev5_gate_id']==3) & (df["ls1"]==1)), "10_3_3"] = 1
    # 11-4-4
    df.loc[((df["gate_id"]==11) & (df['prev_gate_id']==4) & (df['prev2_gate_id']==4)) |
        ((df["gate_id"]==11) & (df['prev_gate_id']==4) & (df['prev3_gate_id']==4)) |
        ((df["gate_id"]==11) & (df['prev2_gate_id']==4) & (df['prev3_gate_id']==4)) |
        ((df["gate_id"]==11) & (df['prev_gate_id']==4) & (df['prev4_gate_id']==4)) |
        ((df["gate_id"]==11) & (df['prev2_gate_id']==4) & (df['prev4_gate_id']==4)) |
        ((df["gate_id"]==11) & (df['prev_gate_id']==4) & (df['prev5_gate_id']==4) & (df["ls1"]==1)), "11_4_4"] = 1 
    df['7_9_5_10'] = (df['gate_id'].isin([7,9,5,10])).astype(int)
    df['9_5_10'] = (df['gate_id'].isin([9,5,10])).astype(int) 
    # 3-11-4     
    df.loc[((df["gate_id"]==3) & (df['prev_gate_id']==11) & (df['prev2_gate_id']==4)) |
        ((df["gate_id"]==3) & (df['prev_gate_id']==11) & (df['prev3_gate_id']==4)) |
        ((df["gate_id"]==3) & (df['prev2_gate_id']==11) & (df['prev3_gate_id']==4)) |
        ((df["gate_id"]==3) & (df['prev_gate_id']==11) & (df['prev4_gate_id']==4)) |
        ((df["gate_id"]==3) & (df['prev2_gate_id']==11) & (df['prev4_gate_id']==4)) |
        ((df["gate_id"]==3) & (df['prev_gate_id']==11) & (df['prev5_gate_id']==4) & (df["ls1"]==1)), "3_11_4"] = 1
    # get previous gate without fillna -2
    df["gs1"] = df["gate_id"].shift(1) 
    df["gs2"] = df["gate_id"].shift(2)
    df["gs3"] = df["gate_id"].shift(3)
    df["gs4"] = df["gate_id"].shift(4)
    df["gs5"] = df["gate_id"].shift(5)
    # get next gate
    df["gs-1"] = df["gate_id"].shift(-1)
    df["gs-2"] = df["gate_id"].shift(-2)
    df["gs-3"] = df["gate_id"].shift(-3)
    df["gs-4"] = df["gate_id"].shift(-4)
    df["gs-5"] = df["gate_id"].shift(-5)  
    # 11-4-4 other realisation
    df.loc[(df["gate_id"].eq(11) & df["gs-1"].eq(4) & df["gs-2"].eq(4)) |
             (df["gate_id"].eq(4) & df["gs1"].eq(11) & df["gs-1"].eq(4) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(4) & df["gs1"].eq(4) & df["gs2"].eq(11) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(11) & df["gs-2"].eq(4) & df["gs-3"].eq(4)) |
             (df["gate_id"].eq(4) & df["gs2"].eq(11) & df["gs-1"].eq(4) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(4) & df["gs1"].eq(4) & df["gs3"].eq(11) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(11) & df["gs-1"].eq(4) & df["gs-3"].eq(4)) |
             (df["gate_id"].eq(4) & df["gs1"].eq(11) & df["gs-2"].eq(4) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(4) & df["gs2"].eq(4) & df["gs3"].eq(11) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(11) & df["gs-3"].eq(4) & df["gs-4"].eq(4)) |
             (df["gate_id"].eq(4) & df["gs3"].eq(11) & df["gs-1"].eq(4) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(4) & df["gs1"].eq(4) & df["gs4"].eq(11) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(11) & df["gs-2"].eq(4) & df["gs-4"].eq(4)) |
             (df["gate_id"].eq(4) & df["gs2"].eq(11) & df["gs-2"].eq(4) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(4) & df["gs2"].eq(4) & df["gs4"].eq(11) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(11) & df["gs-1"].eq(4) & df["gs-4"].eq(4)) |
             (df["gate_id"].eq(4) & df["gs1"].eq(11) & df["gs-3"].eq(4) & df["ls1"].eq(1)) |
             (df["gate_id"].eq(4) & df["gs3"].eq(4) & df["gs4"].eq(11) & df["ls1"].eq(1)), "1144"] = 1
    # 7-3-3-10
    df["73310"] = df["gate_id"][(df["gate_id"].eq(7) & df["gs-1"].eq(3) & df["gs-2"].eq(3) & df["gs-3"].eq(10)) |
                                   (df["gate_id"].eq(3) & df["gs1"].eq(7) & df["gs-1"].eq(3) & df["gs-2"].eq(10)) |
                                   (df["gate_id"].eq(3) & df["gs1"].eq(3) & df["gs2"].eq(7) & df["gs-1"].eq(10)) |
                                   (df["gate_id"].eq(10) & df["gs1"].eq(3) & df["gs2"].eq(3) & df["gs3"].eq(7))].apply(lambda x: 1 if x >= 1 else 0)
    # 9-5-10-3
    df["95103"] = df["gate_id"][(df["gate_id"].eq(9) & df["gs-1"].eq(5) & df["gs-2"].eq(10) & df["gs-3"].eq(3)) |
                                   (df["gate_id"].eq(5) & df["gs1"].eq(9) & df["gs-1"].eq(10) & df["gs-2"].eq(3)) |
                                   (df["gate_id"].eq(10) & df["gs1"].eq(5) & df["gs2"].eq(9) & df["gs-1"].eq(3)) |
                                   (df["gate_id"].eq(3) & df["gs1"].eq(10) & df["gs2"].eq(5) & df["gs3"].eq(9))].apply(lambda x: 1 if x >= 1 else 0)

    # binary for each next gate_id
    for i in range(-1,17):
        df[f'gs1_{i}'] = (df['gs-1'] == i).astype('int')
    
    
    return df

In [5]:
# users that have small or old history
very_wrong_user = [4,20,31,38,44,51,52]
not_in_user = [13,16]
wrong_user = [4,5,7,8,10,20,21,30,31,38,40,42,44,45,51,52,57]

In [6]:
df_train = get_feats(df_train)
df_test = get_feats(df_test)

In [7]:
cols_scale = ['row_id','10_3_3','73310','minute','second',
         'delta_seconds2','delta_seconds3','delta_seconds4','delta_seconds5',
         'ls1','11_4_4','gs-2','gs-4',"gs1",'gs2','gs3',]

In [8]:
scaler = StandardScaler()
scaler.fit(df_train[cols_scale])
df_train[cols_scale] = scaler.transform(df_train[cols_scale])
df_test[cols_scale] = scaler.transform(df_test[cols_scale])

In [9]:
cols = ['row_id','hour','minute','second','start_month','end_month','min_gate_id','max_gate_id','first_gate_id','last_gate_id','moda_gate_id',
         'cnt_gateday','same_gate_id','same2_gate_id','same3_gate_id','same4_gate_id','same5_gate_id','dayofweek',
         'delta_minutes','delta_seconds','delta_seconds2','delta_seconds3','delta_seconds4','delta_seconds5','day',
         'last_week','first_week','weekend','3_4_7_10_11','4_5_7_10_11','3_4','10_11','4_5','delta_seconds_0',
         'delta_seconds_86000','morning','evening','minute15','minute45','delta_minutes45','2days','Monday','Tuesday','Wednesday',
         'Thursday','Friday','Saturday','gate_3', 'gate_4', 'gate_5', 'gate_6', 'gate_7', 'gate_8', 'gate_9','gate_10', 'gate_11', 
         'gate_12','gate_13', 'gate_15','hour_7', 'hour_8', 'hour_9', 'hour_10','hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 
         'hour_16','hour_17', 'hour_18', 'hour_19','hour_20', 'hour_21', 'hour_22','hour_23','prev_gate_3', 'prev_gate_4', 'prev_gate_5',
         'prev_gate_6', 'prev_gate_7', 'prev_gate_8', 'prev_gate_9','prev_gate_10', 'prev_gate_11', 'prev_gate_12', 'prev_gate_13',
         'prev_gate_15','prev2_gate_3', 'prev2_gate_4', 'prev2_gate_5', 'prev2_gate_6', 'prev2_gate_7',
         'prev2_gate_8', 'prev2_gate_9','prev2_gate_10', 'prev2_gate_11', 'prev2_gate_12', 'prev2_gate_13', 'prev2_gate_15',
         'delta_hours_0','delta_hours_1','delta_hours_2','delta_hours_3','delta_hours_4',
         'delta_hours_5','delta_hours_6','delta_hours_7','delta_hours_8','delta_hours_9','delta_hours_10','delta_hours_11',
         'delta_hours_12','delta_hours_13','delta_hours_14','delta_hours_15','delta_hours_16','delta_hours_17','delta_hours_18',
         'delta_hours_19','delta_hours_20','delta_hours_21','delta_hours_22','delta_hours_23','lag','l3s','l-1','ls1','lag1','lag2','lag3',
         'lag4','lag5','lag6','lag7','lag8','lag9','lag0','lagx','double','double2','double3','double4','double5','double6','double7',
         'double8','10_3_3','11_4_4','7_9_5_10','9_5_10','delta_seconds3_86000','gs-1','gs-2','gs-3','gs-4',
         'gs-5','1144','73310','gs1_3','gs1_4','gs1_5','gs1_6','gs1_7','gs1_8','gs1_9','gs1_10','gs1_11',
         'gs1_12','gs1_13','gs1_14','gs1_15','gs1_16',"gs1",'gs2','gs3','gs4','gs5',"3_11_4"]

In [10]:
# drop strange users
df_train = df_train[~(df_train['user_id'].isin(wrong_user))]
df_test = df_test[~(df_test['user_id'].isin(wrong_user))]
df_test = df_test[~(df_test['user_id'].isin([2,41,43]))]

In [11]:
d = {0:0, 1:1, 3:2,  6:3,  9:4, 11:5, 12:6, 14:7, 15:8, 17:9, 18:10, 19:11, 22:12, 23:13, 24:14, 25:15, 26:16, 27:17, 28:18, 29:19,
     32:20, 33:21, 34:22, 35:23,36:24, 37:25, 39:26, 46:27, 47:28, 48:29, 49:30, 50:31, 53:32, 54:33, 55:34, 56:35}
y_train = df_train['user_id'].map(d)
y_test = df_test['user_id'].map(d)

In [12]:
clf = XGBClassifier(objective='multi:softmax', 
                            gamma=0.1,
                            learning_rate=0.1,
                            max_depth=7,
                            reg_lambda=0.5,
                            subsample=0.8, 
                            eval_metric=['merror'],
                            seed=42)
clf.fit(df_train[cols].fillna(0), y_train)
accuracy_score(y_train, clf.predict(df_train[cols].fillna(0))) * 100,\
accuracy_score(y_test, clf.predict(df_test[cols].fillna(0))) * 100

(90.41767100011855, 18.716006216006217)

In [13]:
(90.41767100011855, 18.716006216006217)

(90.41767100011855, 18.716006216006217)

In [14]:
# check if without i column metric is higher
# for i in cols:
#     clf.fit(df_train[cols].fillna(0).drop(i, axis=1), y_train)
#     score_train = accuracy_score(y_train, clf.predict(df_train[cols].fillna(0).drop(i, axis=1))) * 100
#     score_val = accuracy_score(y_test, clf.predict(df_test[cols].fillna(0).drop(i, axis=1))) * 100
#     if score_val >= 18.6:
#         print(i, score_train, score_val)
#     else:
#         print(i)    

# all model

In [15]:
df = get_feats(df)

In [16]:
scaler = StandardScaler()
scaler.fit(df[cols_scale])
df[cols_scale] = scaler.transform(df[cols_scale])

In [17]:
df = df[~(df['user_id'].isin(wrong_user))]

In [18]:
d1 = {0:0, 1:1, 2:2, 3:3,  6:4,  9:5, 11:6, 12:7, 14:8, 15:9, 17:10, 18:11, 19:12, 22:13, 23:14, 24:15, 25:16, 26:17, 27:18, 28:19, 29:20,
     32:21, 33:22, 34:23, 35:24,36:25, 37:26, 39:27,41:28,43:29, 46:30, 47:31, 48:32, 49:33, 50:34, 53:35, 54:36, 55:37, 56:38}
y = df['user_id'].map(d1)

In [19]:
clf = XGBClassifier(objective='multi:softmax', 
                            gamma=0.1, 
                            learning_rate=0.1,
                            max_depth=7,
                            reg_lambda=0.5,
                            subsample=0.8, 
                            eval_metric=['merror'],
                            seed=42)
clf.fit(df[cols].fillna(0), y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=['merror'],
              feature_types=None, gamma=0.1, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', predictor=None, ...)

In [20]:
test = pd.read_csv('test.csv')
test = test.sort_values('timestamp')
test['row_id_1'] = test['row_id'] # because standard scaller on row_id rewrite it
test = get_feats(test)

In [21]:
# based on EDA
test['user_id'] = np.where(test['gate_id'] == 14, 12, 999)
test['user_id'] = np.where(test['gate_id'] == 1, 25, test['user_id'])
test['user_id'] = np.where(test['gate_id'] == -1, 46, test['user_id'])

In [22]:
test[cols_scale] = scaler.transform(test[cols_scale])
pred = clf.predict(test[cols].fillna(0))
test['pred'] = pred
test['pred'] = test['pred'].map({i:j for i,j in zip(d1.values(), d1.keys())})

In [23]:
test['row_id'] = test['row_id_1'] # return original row_id back

In [24]:
test['user_id'] = np.where(test['user_id'] == 999, test['pred'], test['user_id'])

In [25]:
sub = pd.read_csv('sample_submission.csv')
sub = sub.drop('target',axis=1).merge(test[['user_id','row_id']], how='left', on='row_id')
sub.columns = ['row_id', 'target']

In [26]:
sub.head(10)

,row_id,target
0,37518,3
1,37519,3
2,37520,3
3,37521,3
4,37522,3
5,37523,1
6,37524,1
7,37525,1
8,37526,1
9,37527,1


In [27]:
sub.to_csv('xgb26_fin.csv',index=False)

0.2810218978 on public leaderboard   
0.269716531 on private leaderboard